In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

tz = pytz.timezone('Europe/Berlin')

dst_start= {
    2019: datetime(2019, 3, 31, tzinfo=tz),
    2020: datetime(2020, 3, 29, tzinfo=tz),
    2021: datetime(2021, 3, 28, tzinfo=tz),
    2022: datetime(2022, 3, 27, tzinfo=tz),
    2023: datetime(2023, 3, 26, tzinfo=tz),
    2024: datetime(2024, 3, 31, tzinfo=tz),
    2025: datetime(2025, 3, 30, tzinfo=tz)
}

dst_end = {
    2019: datetime(2019, 10, 27, tzinfo=tz),
    2020: datetime(2020, 10, 25, tzinfo=tz),
    2021: datetime(2021, 10, 31, tzinfo=tz),
    2022: datetime(2022, 10, 30, tzinfo=tz),
    2023: datetime(2023, 10, 29, tzinfo=tz),
    2024: datetime(2024, 10, 27, tzinfo=tz),
    2025: datetime(2025, 10, 26, tzinfo=tz)
}


In [21]:
columns = ['Delivery day'] + \
              [f'Hour {i}' for i in range(1, 3)] + \
              ['Hour 3A', 'Hour 3B'] + \
              [f'Hour {i}' for i in range(4, 25)]


In [43]:
import os
import shutil
base_folder_name = 'daa_market'
if os.path.exists(base_folder_name):
    shutil.rmtree(base_folder_name)
os.mkdir(base_folder_name)


In [44]:
for year in dst_start.keys():
    os.mkdir(os.path.join(base_folder_name, str(year)))
    dates = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='D', tz=tz)
    for month in range(1, 13):
        month_dates = dates[dates.month == month]
        os.mkdir(os.path.join(base_folder_name, str(year), str(month)))
        data = []
        for date in month_dates:
        # Generate random prices for all hours (0-100)
            prices = np.round(np.random.uniform(0, 100, 25), 2)
            prices_dict = {col: price for col, price in zip(columns[1:], prices)}

            # Handle DST transitions
            if date.date() == dst_start[year].date():
                # Spring forward (23h day) - skip Hour3A and Hour3B
                prices_dict['Hour 3A'] = np.nan
                prices_dict['Hour 3B'] = np.nan
            elif date.date() == dst_end[year].date():
                # Fall back (25h day) - keep both Hour3A and Hour3B
                pass
            else:
                # Normal day (24h) - keep Hour3A, skip Hour3B
                prices_dict['Hour 3B'] = np.nan

            # Create row
            row = {'Delivery day': date.strftime('%d/%m/%Y')}
            row.update(prices_dict)
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        df.to_csv(os.path.join(base_folder_name, str(year), str(month),'daa_market.csv'), index=False)
